In [1]:
# ==============================================================
# 1) Setup & imports — IoU between DEM_SW and GEE_SW monthly vectors
# ==============================================================

# If needed in your ArcGIS Pro conda env:
# !pip install -q geopandas shapely pyproj pandas numpy tqdm matplotlib

from pathlib import Path
import re
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass

import numpy as np
import pandas as pd
from tqdm import tqdm

import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon, MultiPolygon
from shapely.ops import unary_union
from shapely import geometry as sgeom

from pyproj import CRS, Transformer

import matplotlib.pyplot as plt


In [2]:
# ==============================================================
# 2) User settings — paths, options, filters, outputs
# ==============================================================

ROOT = Path(r"C:\Users\ibana\Desktop\JRC_Tanganica\GIS_Intermediate\Intermediate_files\SurfaceWater")

# Inputs
VEC_DIR     = ROOT / "vectors"
DEM_SW_DIR  = VEC_DIR / "DEM_SW"          # subfolders e.g., 'Tanganyika_16', 'Kivu_162'
GEE_SW_DIR  = VEC_DIR / "GEE_SW"          # subfolders 'Tanganyika', 'Kivu'

# Optional AOI files (true outlines), useful for sanity/plots (not required for IoU)
AOI_DIR = ROOT / "aoi"
AOI_FILES = {
    "Tanganyika": AOI_DIR / "HydroLAKES_polys_v10_Tanganyika.shp",
    "Kivu":       AOI_DIR / "HydroLAKES_polys_v10_Kivu.shp",
}

# Outputs
IOU_ROOT    = ROOT / "IoU"
STATS_DIR   = IOU_ROOT / "stats"
PLOTS_DIR   = STATS_DIR / "plots"
TABLES_DIR  = STATS_DIR / "tables"
DIFFS_DIR   = IOU_ROOT / "diffs"     # difference polygons (optional)
for d in [IOU_ROOT, STATS_DIR, PLOTS_DIR, TABLES_DIR, DIFFS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# Lakes to process
LAKES = ["Tanganyika", "Kivu"]

# Pairing/testing
RUN_ONLY_YYYY_MM = None   # e.g., "2024_07" to test a single month; None=all matched months

# Geometry handling
DISSOLVE_INPUTS     = True          # dissolve each input file to a single (multi)polygon
MIN_VALID_AREA_M2   = 1.0           # drop degenerate geometries after projection

# Boundary diagnostics (computed in LAEA meters)
BOUNDARY_DIAGNOSTICS = True
N_BOUNDARY_POINTS    = 1500         # total samples per geometry boundary (reduce if slow)
RANDOM_SEED          = 42

# Difference polygons export (in LAEA meters)
EXPORT_DIFFS = True

# Plotting
MAKE_PLOTS = True

# Filename patterns
DEM_FILE_REGEX = re.compile(r".*_(\d{6}).*_diss\.(shp|gpkg|geojson)$", re.IGNORECASE)  # DEM
GEE_FILE_REGEX = re.compile(r"^sw_(\d{4})_(\d{2})_(Tanganyika|Kivu)\.(shp|gpkg|geojson)$", re.IGNORECASE)  # GEE


In [3]:
# ==============================================================
# 3) Helpers — parsing, CRS normalization, LAEA setup, unions
# ==============================================================

def parse_yymm_from_dem(path: Path) -> Optional[str]:
    # Tanganyika_16_202503_mean_..._diss.shp → '2025_03'
    m = DEM_FILE_REGEX.match(path.name)
    if not m: return None
    yyyymm = m.group(1)
    return f"{yyyymm[:4]}_{yyyymm[4:6]}"

def parse_yymm_from_gee(path: Path) -> Optional[str]:
    # sw_2025_03_Tanganyika.gpkg → '2025_03'
    m = GEE_FILE_REGEX.match(path.name)
    if not m: return None
    yyyy, mm = m.group(1), m.group(2)
    return f"{yyyy}_{mm}"

def ensure_wgs84(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    Normalize to WGS84 for consistent centroiding / LAEA setup.
    If CRS is missing, we assume it's already lon/lat; a warning is printed.
    """
    if gdf.crs is None:
        print(f"[warn] Missing CRS in {getattr(gdf, '__geo_interface__', 'GeoDataFrame')}; assuming EPSG:4326.")
        gdf = gdf.set_crs("EPSG:4326", allow_override=True)
    else:
        gdf = gdf.to_crs("EPSG:4326")
    return gdf

def local_laea_crs(geomA: sgeom.base.BaseGeometry, geomB: sgeom.base.BaseGeometry) -> CRS:
    """
    Build a local Lambert Azimuthal Equal-Area centered at the centroid of A∪B (lon/lat).
    """
    combo = unary_union([geomA, geomB])
    c = combo.centroid
    lat0, lon0 = c.y, c.x
    proj4 = f"+proj=laea +lat_0={lat0} +lon_0={lon0} +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs"
    return CRS.from_proj4(proj4)

def gs_union_all(gseries: gpd.GeoSeries):
    """
    Safe union preferring GeoSeries.union_all() (Shapely/GeoPandas ≥2.0),
    falling back to unary_union on older stacks.
    """
    try:
        return gseries.union_all()
    except Exception:
        return unary_union(gseries)


In [4]:
# ==============================================================
# 4) Inventory & pairing — map months to DEM & GEE files (per lake)
# ==============================================================

def find_dem_files_for_lake(lake: str) -> Dict[str, Path]:
    mapping = {}
    for sub in DEM_SW_DIR.glob(f"{lake}_*"):
        if not sub.is_dir(): continue
        for ext in ("*.shp", "*.gpkg", "*.geojson"):
            for p in sub.glob(ext):
                if not p.name.lower().endswith("_diss" + p.suffix.lower()):
                    continue
                key = parse_yymm_from_dem(p)
                if key: mapping[key] = p
    return mapping

def find_gee_files_for_lake(lake: str) -> Dict[str, Path]:
    mapping = {}
    sub = GEE_SW_DIR / lake
    for ext in ("*.shp", "*.gpkg", "*.geojson"):
        for p in sub.glob(ext):
            key = parse_yymm_from_gee(p)
            if key: mapping[key] = p
    return mapping

def pair_months(lake: str) -> List[Tuple[str, Path, Path]]:
    dem_map = find_dem_files_for_lake(lake)
    gee_map = find_gee_files_for_lake(lake)
    keys = sorted(set(dem_map.keys()) & set(gee_map.keys()))
    if RUN_ONLY_YYYY_MM:
        keys = [k for k in keys if k == RUN_ONLY_YYYY_MM]
    return [(k, dem_map[k], gee_map[k]) for k in keys]

# Quick peek
for L in LAKES:
    P = pair_months(L)
    print(f"{L}: matched months = {len(P)} →", [k for k,_,_ in P[:5]])


Tanganyika: matched months = 17 → ['2023_11', '2023_12', '2024_01', '2024_02', '2024_03']
Kivu: matched months = 17 → ['2023_11', '2023_12', '2024_01', '2024_02', '2024_03']


In [5]:
# ==============================================================
# 5) Metrics — IoU, Dice, precision/recall, areas; boundary distances
# ==============================================================

@dataclass
class AreaMetrics:
    yyyy_mm: str
    lake: str
    area_dem_km2: float
    area_gee_km2: float
    inter_km2: float
    union_km2: float
    iou: float             # Jaccard
    dice: float            # F1 = 2*inter/(sum areas)
    precision: float       # inter / area_gee
    recall: float          # inter / area_dem
    abs_diff_km2: float    # |area_gee - area_dem|
    rel_diff_pct: float    # 100*(gee-dem)/dem

@dataclass
class BoundaryMetrics:
    yyyy_mm: str
    lake: str
    nA: int
    nB: int
    mean_m: float
    median_m: float
    p90_m: float
    p95_m: float
    max_m: float

def to_km2(m2: float) -> float:
    return float(m2) / 1e6

def iou_and_friends(A: sgeom.base.BaseGeometry, B: sgeom.base.BaseGeometry) -> Tuple[float,float,float,float]:
    inter = A.intersection(B)
    inter_a = inter.area
    a = A.area
    b = B.area
    union_a = a + b - inter_a
    iou = inter_a / union_a if union_a > 0 else float('nan')
    dice = (2*inter_a) / (a + b) if (a+b) > 0 else float('nan')
    return inter_a, union_a, iou, dice

def densify_boundary_points(geom: sgeom.base.BaseGeometry, n_points=1500) -> List[Point]:
    """
    Sample ~n_points along the exterior boundaries, proportional to ring lengths.
    """
    if geom.is_empty: return []
    geoms = [geom] if isinstance(geom, Polygon) else list(geom.geoms)
    lines = []
    for g in geoms:
        if isinstance(g, Polygon) and not g.is_empty:
            lines.append(LineString(g.exterior.coords))
    if not lines: return []
    lengths = np.array([ln.length for ln in lines], dtype=float)
    Ltot = float(lengths.sum())
    if Ltot <= 0: return []
    n_per = np.maximum(1, np.round(n_points * (lengths / Ltot))).astype(int)
    pts = []
    for ln, n in zip(lines, n_per):
        for i in range(n):
            t = (i + 0.5) / n
            pts.append(ln.interpolate(t, normalized=True))
    return pts

def nearest_distances(A_pts: List[Point], B_geom: sgeom.base.BaseGeometry) -> np.ndarray:
    """
    Distance from each sampled point on A's boundary to B's boundary (meters in LAEA).
    Direct GEOS distance avoids STRtree type issues and is fast for ~thousands of points.
    """
    if not A_pts or B_geom.is_empty:
        return np.array([], dtype=float)
    B_bnd = B_geom.boundary
    return np.array([p.distance(B_bnd) for p in A_pts], dtype=float)

def summarize_distances(dAB: np.ndarray, dBA: np.ndarray, yyyy_mm: str, lake: str) -> BoundaryMetrics:
    if dAB.size == 0 and dBA.size == 0:
        return BoundaryMetrics(yyyy_mm, lake, 0, 0, *(float('nan'),)*5)
    both = np.concatenate([dAB, dBA]) if dAB.size and dBA.size else (dAB if dBA.size==0 else dBA)
    return BoundaryMetrics(
        yyyy_mm, lake,
        int(dAB.size), int(dBA.size),
        float(np.mean(both)), float(np.median(both)),
        float(np.percentile(both, 90)), float(np.percentile(both, 95)),
        float(np.max(both))
    )


In [6]:
# ==============================================================
# 6) IoU engine — read, project to local LAEA, compute metrics, export diffs
# ==============================================================

def compute_metrics_for_pair(yyyy_mm: str, lake: str, dem_path: Path, gee_path: Path) -> Tuple[AreaMetrics, Optional[BoundaryMetrics]]:
    # Read and normalize to WGS84 for LAEA setup
    dem_gdf = gpd.read_file(dem_path); dem_wgs = ensure_wgs84(dem_gdf)
    gee_gdf = gpd.read_file(gee_path); gee_wgs = ensure_wgs84(gee_gdf)

    # Dissolve to single geometry per side, fix minor invalidities
    dem_union_wgs = gs_union_all(dem_wgs.geometry).buffer(0)
    gee_union_wgs = gs_union_all(gee_wgs.geometry).buffer(0)
    if dem_union_wgs.is_empty or gee_union_wgs.is_empty:
        am = AreaMetrics(yyyy_mm, lake, *(float('nan'),)*10)
        return am, None

    # Local equal-area CRS for robust area/length/distance in meters
    laea = local_laea_crs(dem_union_wgs, gee_union_wgs)
    dem_laea = dem_wgs.to_crs(laea).geometry
    gee_laea = gee_wgs.to_crs(laea).geometry
    dem_laea = gs_union_all(dem_laea).buffer(0)
    gee_laea = gs_union_all(gee_laea).buffer(0)

    if dem_laea.area < MIN_VALID_AREA_M2 or gee_laea.area < MIN_VALID_AREA_M2:
        am = AreaMetrics(yyyy_mm, lake,
                         to_km2(dem_laea.area), to_km2(gee_laea.area),
                         *(float('nan'),)*6)
        return am, None

    # Core metrics
    inter_m2, union_m2, iou, dice = iou_and_friends(dem_laea, gee_laea)
    area_dem_km2 = to_km2(dem_laea.area)
    area_gee_km2 = to_km2(gee_laea.area)
    inter_km2    = to_km2(inter_m2)
    union_km2    = to_km2(union_m2)
    precision    = inter_m2 / gee_laea.area if gee_laea.area > 0 else float('nan')
    recall       = inter_m2 / dem_laea.area if dem_laea.area > 0 else float('nan')
    abs_diff     = abs(area_gee_km2 - area_dem_km2)
    rel_diff     = ( (area_gee_km2 - area_dem_km2) / area_dem_km2 * 100.0 ) if area_dem_km2 > 0 else float('nan')

    am = AreaMetrics(yyyy_mm, lake, area_dem_km2, area_gee_km2,
                     inter_km2, union_km2, float(iou), float(dice),
                     float(precision), float(recall), float(abs_diff), float(rel_diff))

    # Boundary distances
    bm = None
    if BOUNDARY_DIAGNOSTICS:
        np.random.seed(RANDOM_SEED)
        ptsA = densify_boundary_points(dem_laea, n_points=N_BOUNDARY_POINTS)
        ptsB = densify_boundary_points(gee_laea, n_points=N_BOUNDARY_POINTS)
        dAB = nearest_distances(ptsA, gee_laea)  # DEM → GEE
        dBA = nearest_distances(ptsB, dem_laea)  # GEE → DEM
        bm  = summarize_distances(dAB, dBA, yyyy_mm, lake)

    # Optional: export difference polygons for visual QA
    if EXPORT_DIFFS:
        out_gp = (DIFFS_DIR / lake)
        out_gp.mkdir(parents=True, exist_ok=True)
        out_file = out_gp / f"diff_{lake}_{yyyy_mm}.gpkg"
        # Build layers in LAEA (meters)
        gpd.GeoDataFrame(geometry=[dem_laea], crs=laea).to_file(out_file, driver="GPKG", layer="dem")
        gpd.GeoDataFrame(geometry=[gee_laea], crs=laea).to_file(out_file, driver="GPKG", layer="gee")
        gpd.GeoDataFrame(geometry=[dem_laea.intersection(gee_laea)], crs=laea).to_file(out_file, driver="GPKG", layer="intersection")
        gpd.GeoDataFrame(geometry=[dem_laea.difference(gee_laea)], crs=laea).to_file(out_file, driver="GPKG", layer="dem_minus_gee")
        gpd.GeoDataFrame(geometry=[gee_laea.difference(dem_laea)], crs=laea).to_file(out_file, driver="GPKG", layer="gee_minus_dem")
        gpd.GeoDataFrame(geometry=[dem_laea.union(gee_laea)], crs=laea).to_file(out_file, driver="GPKG", layer="union")

    return am, bm


In [ ]:
# ==============================================================
# 7) Run — iterate lakes & matched months; save tables; (optional) plots
# ==============================================================

all_area_rows, all_bound_rows = [], []

for lake in LAKES:
    pairs = pair_months(lake)
    if not pairs:
        print(f"[warn] No matched months for {lake}")
        continue

    area_rows, bound_rows = [], []

    for yyyy_mm, dem_p, gee_p in tqdm(pairs, desc=f"IoU {lake}", unit="month"):
        am, bm = compute_metrics_for_pair(yyyy_mm, lake, dem_p, gee_p)
        area_rows.append(am.__dict__)
        if bm is not None:
            bound_rows.append(bm.__dict__)

    # Per-lake tables
    area_df = pd.DataFrame(area_rows).sort_values("yyyy_mm")
    area_csv = TABLES_DIR / f"iou_area_{lake}.csv"
    area_df.to_csv(area_csv, index=False, encoding="utf-8-sig")

    if BOUNDARY_DIAGNOSTICS and bound_rows:
        bound_df = pd.DataFrame(bound_rows).sort_values("yyyy_mm")
        bound_csv = TABLES_DIR / f"boundary_{lake}.csv"
        bound_df.to_csv(bound_csv, index=False, encoding="utf-8-sig")
    else:
        bound_df, bound_csv = pd.DataFrame(), None

    print(f"[done] {lake}: saved {area_csv}" + (f" and {bound_csv}" if bound_csv else ""))

    # Accumulate global
    all_area_rows.extend(area_rows)
    all_bound_rows.extend(bound_rows)

# Global tables
if all_area_rows:
    glob_area_df  = pd.DataFrame(all_area_rows).sort_values(["lake","yyyy_mm"])
    glob_area_csv = TABLES_DIR / "iou_area_all.csv"
    glob_area_df.to_csv(glob_area_csv, index=False, encoding="utf-8-sig")
    print("[global] area table →", glob_area_csv)

if all_bound_rows:
    glob_bound_df  = pd.DataFrame(all_bound_rows).sort_values(["lake","yyyy_mm"])
    glob_bound_csv = TABLES_DIR / "boundary_all.csv"
    glob_bound_df.to_csv(glob_bound_csv, index=False, encoding="utf-8-sig")
    print("[global] boundary table →", glob_bound_csv)


IoU Tanganyika:  71%|███████   | 12/17 [31:58<10:22, 124.55s/month] 

In [ ]:
# ==============================================================
# 8) Plots — IoU/Dice, Precision/Recall, Areas (saved to stats/plots)
# ==============================================================

if MAKE_PLOTS and all_area_rows:
    area_df = pd.DataFrame(all_area_rows).sort_values(["lake","yyyy_mm"])
    area_df["date"] = pd.to_datetime(area_df["yyyy_mm"] + "-01")

    for lake in LAKES:
        sub = area_df[area_df["lake"] == lake].sort_values("date")
        if sub.empty: continue

        # IoU & Dice
        plt.figure(figsize=(9,3.6))
        plt.plot(sub["date"], sub["iou"], marker="o", linewidth=1, label="IoU (Jaccard)")
        plt.plot(sub["date"], sub["dice"], marker="s", linewidth=1, label="Dice (F1)")
        plt.title(f"{lake}: IoU (Jaccard) and Dice over time")
        plt.xlabel("Date"); plt.ylabel("Score")
        plt.grid(True, linestyle="--", linewidth=0.5); plt.legend()
        plt.tight_layout()
        plt.savefig(PLOTS_DIR / f"{lake}_iou_dice.png", dpi=150)
        plt.close()

        # Precision & Recall
        plt.figure(figsize=(9,3.6))
        plt.plot(sub["date"], sub["precision"], marker="o", linewidth=1, label="Precision (GEE vs DEM)")
        plt.plot(sub["date"], sub["recall"],    marker="s", linewidth=1, label="Recall (DEM covered by GEE)")
        plt.title(f"{lake}: Precision & Recall over time")
        plt.xlabel("Date"); plt.ylabel("Score")
        plt.grid(True, linestyle="--", linewidth=0.5); plt.legend()
        plt.tight_layout()
        plt.savefig(PLOTS_DIR / f"{lake}_precision_recall.png", dpi=150)
        plt.close()

        # Areas
        plt.figure(figsize=(9,3.6))
        plt.plot(sub["date"], sub["area_dem_km2"], marker="o", linewidth=1, label="DEM area (km²)")
        plt.plot(sub["date"], sub["area_gee_km2"], marker="s", linewidth=1, label="GEE area (km²)")
        plt.title(f"{lake}: Monthly areas")
        plt.xlabel("Date"); plt.ylabel("Area (km²)")
        plt.grid(True, linestyle="--", linewidth=0.5); plt.legend()
        plt.tight_layout()
        plt.savefig(PLOTS_DIR / f"{lake}_areas.png", dpi=150)
        plt.close()

    print("[plots] Saved to", PLOTS_DIR)


In [ ]:
# ==============================================================
# 9) Console summary — report-friendly quick stats
# ==============================================================

if all_area_rows:
    df = pd.DataFrame(all_area_rows)
    for lake in LAKES:
        sub = df[df["lake"] == lake].copy()
        if sub.empty: continue
        print(f"\n=== {lake} summary ===")
        print(f"Months matched: {len(sub)}")
        print("IoU:   mean={:.3f}  median={:.3f}  min={:.3f}  max={:.3f}".format(
            sub["iou"].mean(), sub["iou"].median(), sub["iou"].min(), sub["iou"].max()))
        print("Dice:  mean={:.3f}  median={:.3f}".format(sub["dice"].mean(), sub["dice"].median()))
        print("Prec:  mean={:.3f}  Recall: mean={:.3f}".format(sub["precision"].mean(), sub["recall"].mean()))
        print("Areas (km²): DEM mean={:.1f}, GEE mean={:.1f}".format(
            sub["area_dem_km2"].mean(), sub["area_gee_km2"].mean()))
